# Homework 13 Advanced Pandas 2

## Task 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [61]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')

ratings.head()

aggr = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': ['max','min', lambda x: max(x) - min(x)]})
aggr.loc[aggr['rating']['count'] > 100]['timestamp']['<lambda_0>'].mean()

40080507.4496124

## Task 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [6]:
import pandas as pd

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

# without address
revenue = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer').fillna(0)

# add address
client_base.merge(revenue, on='client_id', how='outer')


,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


## Task 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Начну с ответа на вспомогательные вопросы:

-  Под набором координат, полагаю, подразумевается набор местоположений, определенных приложением в разное время. Как связать их с фактом визита/действия? Рассмотрим следующие варианты:
    1. Координаты идут в паре с timestamp. Если есть timestamp, совпадающий с timestamp визитов/покупок, координаты можно отбирать по равенству этих значений.
    2. Timestamp есть, но не совпадает. В таком случаем можно определять ближайший к визиту/покупке timestamp из набора с координатами и делать допущение о принятии соответствующих координат за искомое значение.
    3. Timestamp отсутствует. В таком случае единственной возможной стратегией мне видится определение, условно говоря, "домашнего региона" пользователя на основании принадлежности большинства координат из набора к определенному региону/местности. И далее как постоянное значение координат используется усредненное значение из набора координат, лежащих в "домашнем регионе".
    
- Удаленность может внести погрешность в расчет, который предлагается в пункте 3 вопроса выше. Наиболее удаленные координаты, заведомо не входящие в "домашний регион", следует игнорировать (как "промахи" в лабораторном эксперименте).

- Из координат можно определить, где территориально находится пользователь в текущий момент/в большинстве случаев. Геопривязка может помочь с определением языка интерфейса, часового пояса, увеличением релевантности выдачи товаров/услуг на основании предположение о том, что жители одного региона имеют наиболее схожие вкусы/потребности.

Таким образом, в объединенный пул визитов/покупок можно добавить столбец с координатами, определенным в соответствии с пунктами 1-3 выше.